## Homework 4 MAE-207

In [1]:
import time
from IPython.display import clear_output
import numpy as np
import matplotlib as mpl

# import sympy
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, atan, pi, sqrt, atan2
from sympy import Matrix
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sympy import lambdify
# import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style

from IPython import display # for the animation
# import matplotlib as mpl
# mpl.use('Qt5Agg')
# plt.ion()
import os
import matplotlib.pyplot as plt
# plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
#from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
#from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

import time

import odrive
from odrive.utils import *
import odrive.enums
%matplotlib qt

## Connect to ODrive

In [2]:
odrv0 = odrive.find_any()
if odrv0 is not None:
    print('Connected!')
    print('Odrive serial {}'.format(odrv0.serial_number))
    
    m0 = odrv0.axis0.motor.is_calibrated
    m1 = odrv0.axis1.motor.is_calibrated
    
    print('Motor 0 calibrated: {}'.format(m0))
    print('Motor 1 calibrated: {}'.format(m1))    
    
else:
    print('Not connected')
    

Connected!
Odrive serial 53194131976757
Motor 0 calibrated: True
Motor 1 calibrated: True


In [ ]:
m0 = odrv0.axis0.motor.is_calibrated
m1 = odrv0.axis1.motor.is_calibrated
    
print('Motor 0 calibrated: {}'.format(m0))
print('Motor 1 calibrated: {}'.format(m1))

In [3]:
# Setting the velocity and calibration current limit
odrv0.axis0.controller.config.vel_limit=2000000
odrv0.axis1.controller.config.vel_limit=2000000
odrv0.axis0.motor.config.calibration_current=20.0
odrv0.axis1.motor.config.calibration_current=20.0


In [4]:
odrv0.axis0.controller.config.vel_gain = 16 / 10000.0
odrv0.axis1.controller.config.vel_gain = 16 / 10000.0
odrv0.axis1.controller.config.pos_gain=44
odrv0.axis1.controller.config.pos_gain=44

odrv0.axis0.controller.config.vel_integrator_gain = 0.001 #/ 10000.0
odrv0.axis1.controller.config.vel_integrator_gain = 0.001 #5.0 / 10000.0


#### Calibrating the Motors

In [ ]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

time.sleep(15)

print('\t Motor 0 calibration result: {} \r\n'.format(odrv0.axis0.motor.is_calibrated), 
      '\t Motor 1 calibration result: {}'.format(odrv0.axis1.motor.is_calibrated))


In [7]:
odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

In [8]:
odrv0.axis0.controller.set_pos_setpoint(0,0,0)
odrv0.axis1.controller.set_pos_setpoint(0,0,0)

In [ ]:
# Function to make the motors return to idle mode 
def motor_idle(): 
    odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
    odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

In [ ]:

(theta_l, 
 theta_r,
 l1, 
 l2,
 w) = symbols(""" theta_l
                         theta_r
                         l1 
                         l2
                         w """ , real = True)

#### Forward Kinematics definition

In [ ]:
A = 2*l2*l1*sin(theta_l)- 2*l1*l2*sin(theta_r)
B = 2*l2*w - 2*l1*l2*cos(theta_r) + 2*l2*l1*cos(theta_l)
C = 2*(l1**2) + w**2 - 2*l1*l1*sin(theta_l)*sin(theta_r) - 2*l1*w*cos(theta_r) + 2*l1*w*cos(theta_l) - 2*l1*l1*cos(theta_l)*cos(theta_r)

theta3_1 = 2*atan((A + sqrt(A**2 + B**2 - C**2))/(B-C))
theta3_2 = 2*atan((A - sqrt(A**2 + B**2 - C**2))/(B-C))

xc_1 = l1*cos(theta_l) + l2*cos(theta3_1) + w/2
xc_1
xc_2 = l1*cos(theta_l) + l2*cos(theta3_2) + w/2
xc_2

yc_1 = l1*sin(theta_l) + l2*sin(theta3_1)
yc_1
yc_2 = l1*sin(theta_l) + l2*sin(theta3_2)
yc_2
#sympy.simplify(theta3_1)
#sympy.simplify(theta3_2)

In [ ]:
FK1 = Matrix([[xc_1],
             [yc_1]])
FK2 = Matrix([[xc_2],
             [yc_2]])

FK_rtheta = Matrix([[sqrt(xc_1**2 + yc_1**2)],
                   [atan2(yc_1,xc_1)]])
FK_rtheta

In [ ]:
FK1_fast = lambdify((theta_l,theta_r,l1,l2,w),FK1)
FK2_fast = lambdify((theta_l,theta_r,l1,l2,w),FK2)

#### Code to record $\theta_R$ and $\theta_L$ to plot ($\theta_R,\theta_L$) workspace

In [ ]:
# Function to setup home position
def autohome():
    print('Bring motors to home and press ENTER')
    input("<enter>")
    R_pos1=odrv0.axis0.encoder.pos_estimate
    L_pos1=odrv0.axis1.encoder.pos_estimate
    print('Bring motors to 90-90 and press ENTER')
    input("<enter>")
    R_pos2=odrv0.axis0.encoder.pos_estimate
    L_pos2=odrv0.axis1.encoder.pos_estimate
    ML_para=(90/(L_pos2-L_pos1))
    MR_para=((-90)/(R_pos2-R_pos1))
    print(ML_para)
    print(MR_para)
    print(L_pos2, L_pos1, R_pos2, R_pos1)
    print("Done")
    return ML_para,MR_para,L_pos1,R_pos1

In [ ]:
cal_para=autohome()

In [ ]:
def enc_pos(L,R):
    L_pos=(np.rad2deg(L )/cal_para[0])+cal_para[2]
    R_pos=((np.rad2deg(R )-180)/cal_para[1])+cal_para[3]
    return L_pos,R_pos

In [ ]:
def get_angles():
    theta_R=((odrv0.axis0.encoder.pos_estimate)-cal_para[3])*(cal_para[1])+180
    theta_L=((odrv0.axis1.encoder.pos_estimate)-cal_para[2])*(cal_para[0])

#     theta_R=(np.around(odrv0.axis0.encoder.pos_estimate)-cal_para[3])*(cal_para[1])+180
#     theta_L=((np.around(odrv0.axis1.encoder.pos_estimate)-cal_para[2])*cal_para[0])
        
    return np.deg2rad(theta_L),np.deg2rad(theta_R)

In [ ]:
def get_vel():
    vel_R=-(odrv0.axis0.encoder.vel_estimate/(2048*4))*(2*np.pi)
    vel_L=-(odrv0.axis1.encoder.vel_estimate/(2048*4))*(2*np.pi)
    
    return np.array([[vel_L],[vel_R]])

In [ ]:
J1 = FK1.jacobian([theta_l,theta_r])
J_rtheta = FK_rtheta.jacobian([theta_l,theta_r])

### Get the Lambda functions for faster computations

In [ ]:

J1_fast = lambdify((theta_l,theta_r,l1,l2,w),J1)

FK1_fast = lambdify((theta_l,theta_r,l1,l2,w),FK1)

J_rtheta_fast = lambdify((theta_l,theta_r,l1,l2,w),J_rtheta)
Frt_fast = lambdify((theta_l,theta_r,l1,l2,w),FK_rtheta)


In [ ]:
link1_length = 0.09;                  # m 
link2_length = 0.16;                  # m
w_length = 0.07; 
x_eq = 0
y_eq = 0.15

# theta_l, theta_r = get_angles()
# theta_r
# fk_xy = FK1_fast(theta_l,theta_r,link1_length,link2_length,w_length)
# fk_xy
# res = IK_5_link(fk_xy[0], fk_xy[1])
# res[0].x[1]

In [ ]:
def IK_5_link(x, y, l1 = link1_length, l2 = link2_length, w = w_length):
    
    def leg_wide(var):
        return np.linalg.norm([var[1] - np.pi, var[0]])
    
    def x_constraint_equation(var):
        # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    
    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                               {"type": "eq", "fun": y_constraint_equation}))
    
    return (res, np.linalg.norm([x_constraint_equation(res.x), y_constraint_equation(res.x)]))

In [ ]:
def get_trajectory(x,y,d,T,N):
    
    theta_points = []
    # Stance
    z = np.polyfit([x[0],x[1]],[y[0],y[1]],1)
    p = np.poly1d(z)
    n_stance = (np.rint(N*d))
    n_stance = n_stance.astype(int)
    x_stancepoints = np.linspace(x[0],x[1],n_stance+1)
    y_stancepoints = p(x_stancepoints)
    
    # Swing
    z = np.polyfit(x,y,2)
    p = np.poly1d(z)
    n_swing = np.rint((N*(1-d)))
    n_swing = n_swing.astype(int)
    x_swingpoints = np.linspace(x[1],x[0],n_swing+1)
    y_swingpoints = p(x_swingpoints)
    
    x_points = np.append(x_stancepoints,x_swingpoints[1:])
    y_points = np.append(y_stancepoints,y_swingpoints[1:])
    
    t_points = np.linspace(0,T,N+1)
    
    for i in range(x_points.shape[0]):
        
        res = IK_5_link(x_points[i],y_points[i])
        theta_points.append([res[0].x[1],res[0].x[0]])
    
    return theta_points,t_points

In [6]:
# Dump Errors
dump_errors(odrv0)
dump_errors(odrv0,True)

axis0
  axis: no error
  motor: no error
  encoder: no error
  controller: no error
axis1
  axis: no error
  motor: no error
  encoder: no error
  controller: no error
axis0
  axis: no error
  motor: no error
  encoder: no error
  controller: no error
axis1
  axis: no error
  motor: no error
  encoder: no error
  controller: no error


In [ ]:
K_spg=np.array([20,50,100,200,300])
damp_c=np.array([0.2,0.3,0.7,1,1.5])
z = np.polyfit(K_spg, damp_c, 4)
q=np.poly1d(z)

## Cartesian Control

In [ ]:
# Cartesian Control

odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

# axis.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL

x_eq=0
y_eq=0.15

kx = 200  # N/m
ky = 0 # N/m
kT = 0.0285 # Nm/A
c  = q(kx)  #1.5 # Nm/rad
Imax=25    #A
res = IK_5_link(x_eq, y_eq)
L_eq,R_eq=enc_pos(res[0].x[0],res[0].x[1])

odrv0.axis0.controller.set_pos_setpoint(R_eq,0,0)
odrv0.axis1.controller.set_pos_setpoint(L_eq,0,0)
time.sleep(1)
t_end = time.time() + 20

while time.time() < t_end:
       
    theta_l, theta_r = get_angles()
    vel = get_vel()

    fk_xy = FK1_fast(theta_l,theta_r,link1_length,link2_length,w_length)
    J_xy = J1_fast(theta_l,theta_r,link1_length,link2_length,w_length)
    x = fk_xy[0]
    y = fk_xy[1]
    mat_xy = np.array([[x[0]-x_eq],
                       [y[0]-y_eq]])
    mat_k = np.array([[-kx,0],
                      [0,-ky]])
    I1 = (J_xy.T @ mat_k @ mat_xy)/kT
    I = I1 -c*(vel)
    if(I[0]>Imax):
        I[0]=Imax
    elif(I[0]<-Imax):
        I[0]=-Imax
    
    if(I[1]>Imax):
        I[1]=Imax
    elif(I[1]<-Imax):
        I[1]=-Imax
    
    odrv0.axis0.controller.set_current_setpoint(-I[1])
    odrv0.axis1.controller.set_current_setpoint(-I[0])
    print(I1)
    clear_output()
motor_idle()    

## Polar Coordinates Control

In [ ]:
# Polar Coord Control (4 & 0.85)

odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL
odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_CLOSED_LOOP_CONTROL

r_eq=0.15
th_eq=75

x_eq=r_eq*np.cos(np.deg2rad(th_eq))
y_eq=r_eq*np.sin(np.deg2rad(th_eq))

kr = 0  # N/m
kth = 4 # N/rad
kT = 0.0285 # Nm/A
c  = 0.85 #q(kr) #0.85 # q(kr)  # Nm/rad
Imax=25    #A
res = IK_5_link(x_eq, y_eq)
L_eq,R_eq=enc_pos(res[0].x[0],res[0].x[1])

odrv0.axis0.controller.set_pos_setpoint(R_eq,0,0)
odrv0.axis1.controller.set_pos_setpoint(L_eq,0,0)
time.sleep(1)
# motor_idle()
t_end = time.time() + 20

while time.time() < t_end:
       
    theta_l, theta_r = get_angles()
    vel = get_vel()

    fk_rt = Frt_fast(theta_l,theta_r,link1_length,link2_length,w_length)
    J_rt = J_rtheta_fast(theta_l,theta_r,link1_length,link2_length,w_length)
    r = fk_rt[0]
    th = fk_rt[1]
    mat_rt = np.array([[r[0]-r_eq],
                       [th[0]-np.deg2rad(th_eq)]])
    mat_k = np.array([[-kr,0],
                      [0,-kth]])
    I1 = (J_rt.T @ mat_k @ mat_rt)/kT
    I = I1 -c*(vel)

    #     np.where(I>25)]
    if(I[0]>Imax):
        I[0]=Imax
    elif(I[0]<-Imax):
        I[0]=-Imax
    
    if(I[1]>Imax):
        I[1]=Imax
    elif(I[1]<-Imax):
        I[1]=-Imax

    odrv0.axis0.controller.set_current_setpoint(-I[1])
    odrv0.axis1.controller.set_current_setpoint(-I[0])
    print(I)
    clear_output()
motor_idle()    